In [1]:
import numpy as np
import pandas as pd

In [7]:
df=pd.read_csv("titanic.csv")
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [9]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [11]:
df=df.drop(columns="Cabin")
df=df.drop(columns="PassengerId")
df=df.drop(columns="Name")
df=df.drop(columns="Ticket")

In [13]:
df=pd.get_dummies(df,columns=["Sex","Embarked"])

In [15]:
df.head(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,34.5,0,0,7.8292,False,True,False,True,False
1,1,3,47.0,1,0,7.0000,True,False,False,False,True
2,0,2,62.0,0,0,9.6875,False,True,False,True,False


In [17]:
df.isnull().sum()

Survived       0
Pclass         0
Age           86
SibSp          0
Parch          0
Fare           1
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [20]:
from sklearn.impute import SimpleImputer
si=SimpleImputer()
df["Age"]=si.fit_transform(df[["Age"]])
df["Fare"]=si.fit_transform(df[["Fare"]])

In [22]:
df.isnull().sum()

Survived      0
Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [24]:
x=df.drop(columns="Survived")
y=df["Survived"]

In [26]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [28]:
df.head(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,34.5,0,0,7.8292,False,True,False,True,False
1,1,3,47.0,1,0,7.0000,True,False,False,False,True
2,0,2,62.0,0,0,9.6875,False,True,False,True,False


In [30]:
df.astype(int)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,34,0,0,7,0,1,0,1,0
1,1,3,47,1,0,7,1,0,0,0,1
2,0,2,62,0,0,9,0,1,0,1,0
3,0,3,27,0,0,8,0,1,0,0,1
4,1,3,22,1,1,12,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
413,0,3,30,0,0,8,0,1,0,0,1
414,1,1,39,0,0,108,1,0,1,0,0
415,0,3,38,0,0,7,0,1,0,0,1
416,0,3,30,0,0,8,0,1,0,0,1


In [32]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)

DecisionTreeClassifier()

In [34]:
y_pred=dt.predict(x_test)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

1.0

In [38]:
param_grid = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [58]:
from sklearn.model_selection import GridSearchCV
dtree_reg = DecisionTreeClassifier(random_state=42) # Initialize a decision tree regressor
grid_search = GridSearchCV(estimator=dtree_reg, param_grid=param_grid, 
                           cv=6, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')


In [60]:
grid_search.fit(x_train, y_train)
best_dtree_reg = grid_search.best_estimator_

Fitting 6 folds for each of 36 candidates, totalling 216 fits


In [61]:
from sklearn.metrics import mean_squared_error
y_pred = best_dtree_reg.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")
print(f"Test RMSE: {rmse}")

Best parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Test RMSE: 0.0


In [64]:
accuracy_score(y_test,y_pred)

1.0